# 4주차 복습 과제

* 데이터는 와인 품질 데이터로 하였습니다.
* 각 모델간의 결정계수 값 차이를 두기 위해 알파값을 임의로 설정했습니다.
* 상황마다 좋은 모델이 다르기 때문에 해당 과제에서 한 모델의 결정계수값이 다른 모델보다 작다고 해서 더 안좋은 모델이 아닙니다!!

## [1] 데이터 로딩

와인 품질 데이터

* 포르투갈 서북쪽의 대서양을 맞닿고 위치한 비뉴 베르드 지방에서 만들어진 와인을 측정한 데이터
* 개인 정보 및 물류 문제로 인해 화학적변수(입력) 및 감각(출력) 변수만 사용가능(예: 포도 종, 와인 브랜드, 와인 판매 가격등에 대한 데이터가 없음)

변수 설명

| 변수 | 설명 |
|------|------|
| fixed acidity | 와인과 관련된 대부분의 비휘발성 산도 |
| volatile acidity | 너무 많으면 불쾌한 식초 맛을 유발할 수 있는, 와인에 함유된 아세트산 |
| citric acid | 와인에 신선함과 풍미를 더해주는 소량으로 발견되는 구연산 |
| residual sugar | 발효가 끝난 후 남은 당도 |
| chlorides | 와인에 있는 염도 |
| free sulfur dioxide | SO2의 자유형태 |
| total sulfur dioxide | SO2의 자유 및 결합형태 |
| density | 물의 밀도 |
| pH | 와인의 산성 염기성 정도 |
| sulfates | 항균등으로 이용하는 와인의 첨가제 |
| alcohol | 와인의 도수 |
| quality | 0부터 10까지 매기는 와인의 점수(output) |

### [1.1] Problem
데이터를 불러와 wine에 담아주세요


In [ ]:
from google.colab import files
files.upload()

In [ ]:
import pandas as pd

# 1) code
wine = pd.read_csv("winequality-red.csv")

wine.head()

## [2] 데이터 전처리

In [ ]:
from sklearn.model_selection import train_test_split

### [2.1] Problem

 1) quality를 제외한 모든 변수를 polynomialfeatures를 이용하여 변수를 늘려주고(bias는 제외)

 2) quality와 다항 변수들을 합쳐 wine에 다시 담아주세요

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# 1)
poly = PolynomialFeatures(include_bias = False)
feature_list = wine.columns.difference(["quality"])
poly.fit(wine[feature_list])
poly_feat = poly.transform(wine[feature_list]) 

In [ ]:
# 2)
poly_feat_df = pd.DataFrame(poly_feat)
wine = pd.concat([wine, poly_feat_df], axis = 1)

### [2.2] Problem
1) X, y로 데이터를 나눠주세요 (y = wine의 퀄리티)

2) train_test_split을 이용하여 train:test = 75:25로 나눠주세요 (random_state = 3)

In [ ]:
# 1)
poly_feat_df = pd.DataFrame(poly_feat)
feature_list = poly_feat_df.columns.difference(["quality"])
X = poly_feat_df[feature_list]
y = wine["quality"]

In [ ]:
# 2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 3)

## [3] Linear Regression

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
linear_model = make_pipeline(StandardScaler(), LinearRegression())
linear_model.fit(X_train, y_train)

In [ ]:
print('학습 데이터 점수: {}'.format(linear_model.score(X_train,y_train)))
print('평가 데이터 점수: {}'.format(linear_model.score(X_test,y_test)))

## [4] Ridge Regression

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

### [4.1] Problem

1) ridge_model 변수에 StandardScaler() -> Ridge()로 이어지는 파이프라인을 만드세요 (Ridge회귀의 알파값을 10으로 설정해주세요)

2) 위에서 만든 모델에 X_train, y_train을 넣어 fit 해주세요

In [ ]:
from sklearn.linear_model import Ridge

# 1)
ridge_model = make_pipeline(StandardScaler(), Ridge(alpha = 10))
# 2) 
ridge_model.fit(X_train, y_train)

In [ ]:
print('학습 데이터 점수: {}'.format(ridge_model.score(X_train,y_train)))
print('평가 데이터 점수: {}'.format(ridge_model.score(X_test,y_test)))

### [4.2] Problem
1) plot_boston_prices함수에서 expected를 x축, predicted를 y축으로 하는 scatterplot을 seaborn으로 그리는 코드를 넣어주세요.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_wine_quality(expected, predicted):
    plt.figure(figsize=(8,4))
    # 1)
    sns.scatterplot(expected, predicted)
    ## 여기까지 
    sns.lineplot([0, 20],[0,20],c='r',linestyle='--')
    plt.xlabel('True quality')
    plt.ylabel('Predicted quality')

### [4.3] Problem
1) expected변수에 y_test, predicted 변수에 ridge_model로 X_test를 예측한 값을 넣어주세요  
2) plot_wine_quality 함수에 expected와 predicted를 넣어서 plot을 그려주세요.

* 데이터가 빨간 선에 가까울수록 잘 예측한 데이터

In [ ]:
# 1)
expected = y_test
predicted = ridge_model.predict(X_test)
# 2) 
plot_wine_quality(expected, predicted)

### [4.4] Problem
1) 위의 ridge_model을 선형회귀와 같도록 파이프 라인을 다시 만드세요

* 릿지회귀의 수식을 생각해보고 Ridge()의 파라미터를 변경하시면 됩니다.

2) ridge_model을 X_train, y_train으로 fit 시키세요

In [ ]:
# 1)
ridge_model = make_pipeline(StandardScaler(), Ridge(alpha = 0))
# 2)
ridge_model.fit(X_train, y_train)
y_pred_train = ridge_model.predict(X_train)
y_pred_test = ridge_model.predict(X_test)

In [ ]:
print('학습 데이터 점수: {}'.format(ridge_model.score(X_train,y_train)))
print('평가 데이터 점수: {}'.format(ridge_model.score(X_test,y_test)))

## [5] Lasso Regression

### [5.1] Problem

1) lasso_model 변수에 StandardScaler() -> Lasso()로 이어지는 파이프라인을 만드세요 (Ridge회귀의 알파값을 0.01로 설정해주세요)

2) 위에서 만든 모델을 X_train, y_train을 넣어 fit 해주세요

In [ ]:
from sklearn.linear_model import Lasso

# 1)
lasso_model = make_pipeline(StandardScaler(), Lasso(alpha = 0.01))

# 2)
lasso_model.fit(X_train, y_train)

In [ ]:
print('학습 데이터 점수: {}'.format(lasso_model.score(X_train,y_train)))
print('평가 데이터 점수: {}'.format(lasso_model.score(X_test,y_test)))

### [5.2] Problem
1) expected변수에 y_test, predicted 변수에 lasso_model로 X_test를 예측한 값을 넣어주세요  
2) plot_wine_quality함수에 expected와 predicted를 넣어서 plot을 그려주세요.

* 데이터가 빨간 선에 가까울수록 잘 예측한 데이터

In [ ]:
# 1)
predicted = lasso_model.predict(X_test)
expected = y_test

# 2)
plot_wine_quality(expected, predicted)

### [5.3] Problem
1) 위의 lass_model을 선형회귀와 같도록 파이프 라인을 다시 만드세요

* 라쏘회귀의 수식을 생각해보고 Lasso()의 파라미터를 변경하시면 됩니다.

2) lasso_model을 X_train, y_train으로 fit 시키세요

In [ ]:
# 1)
lasso_model = make_pipeline(StandardScaler(), Lasso(alpha = 0))

# 2)
lasso_model.fit(X_train, y_train)

## [6] Elastic-Net

### [6.1] Problem

1) elastic_model 변수에 StandardScaler() -> ElasticNet()으로 이어지는 파이프라인을 만드세요 (알파값을 0.1, l1규제의 비율을 0.2로 설정해주세요)

2) 위에서 만든 모델을 X_train, y_train을 넣어 fit 해주세요

In [ ]:
from sklearn.linear_model import ElasticNet

# 1)
elastic_model = make_pipeline(StandardScaler(), ElasticNet(alpha = 0.0001, l1_ratio = 0.2))
# 2)
elastic_model.fit(X_train, y_train)

In [ ]:
print('학습 데이터 점수: {}'.format(elastic_model.score(X_train,y_train)))
print('평가 데이터 점수: {}'.format(elastic_model.score(X_test,y_test)))

### [6.2] Problem

1) elastic_model parameter을 반복문을 통해 alpha별 점수들을 train_score와 test_score에 담아주세요 (알파는 [0.001, 0.01, 0.1, 1. 10, 100,]이고, iteration은 10000번) 

2) 그래프를 통해 다음과 같이 나타내주세요

In [ ]:
train_score=[]
test_score=[]

# 1) 
alpha_list=[0.001,0.01,0.1,1,10,100]
for alpha in alpha_list:
    #엘라스틱 모델
    elasticNet=ElasticNet(alpha=alpha,max_iter=10000)
    #훈련
    elasticNet.fit(X_train, y_train)
    #훈련 점수와 테스트 점수 저장
    train_score.append(elasticNet.score(X_train, y_train))
    test_score.append(elasticNet.score(X_test, y_test))

In [ ]:
import numpy as np

In [ ]:
# 2)
plt.plot(np.log10(alpha_list),train_score, label = "train")
plt.plot(np.log10(alpha_list),test_score, label = "test")
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.legend()
plt.show()

## [7] VIF



### [7.1] Challenge
1) 다음 함수를 완성시키고 다음과 같은 출력이 나오도록 코드를 완성시켜보세요

2) vif가 높은 10개의 변수를 다음과 같이 출력하고

3) X_train에서 나머지 변수들만 이용해서 다시 elasticnet model(alpha = 0.1)에 넣어 평가해보고 vif도 평가해보세요

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 1)
# def vif_df(X):
#   vif = pd.DataFrame()
#   
#   return vif
def vif_df(X):
  vif = pd.DataFrame()
  vif["VIF_Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
  vif["Feature"] = X.columns
  return vif

vif = vif_df(X_train)
vif

In [ ]:
# 2)
print(vif["VIF_Factor"].sort_values(ascending=True)[:11].index)

In [ ]:
# 3)
X_train_vif = X_train.drop(list(vif["VIF_Factor"].sort_values(ascending=True)[:11].index), axis = 1)
X_test_vif = X_test.drop(list(vif["VIF_Factor"].sort_values(ascending=True)[:11].index), axis = 1)

elastic_model = make_pipeline(StandardScaler(), ElasticNet(alpha = 0.1))
elastic_model.fit(X_train_vif, y_train)

print('학습 데이터 점수: {}'.format(elastic_model.score(X_train_vif,y_train)))
print('평가 데이터 점수: {}'.format(elastic_model.score(X_test_vif,y_test)))

vif_df(X_train_vif).sort_values(["VIF_Factor"], ascending = True)

## [8] 서술형 문항

### [8-1] Problem
릿지회귀 라쏘회귀의 차이점과 각각의 장단점을 입력해주세요

답 : 
차이점 :
 릿지는 제곱의 형태 라쏘는 절댓값의 형태로 규제를 한다.

장점 :
과적합이 될경우 variance를 낮춰져서 train-validation간의 성능차를 좁혀준다. 모델의 성능을 평가(test set)할 때 효과적일 수 있다.


단점 :
애초에 모델이 단순할 경우( 변수의 개수가 적은 경우) 라쏘나 릿지회귀를 통해서 할 경우 과소적합이 되어 bias가 높아져서 대체적인 모델의 성능이 낮아질 수 있다.

### [8-2] Problem
보다시피 r^2의 값이 0.3~0.5로 매우 낮게 평가되었습니다. 그 이유가 무엇이라고 생각하시나요?

답 : 
이는 아마 데이터 형태자체가 비선형 모델이기 때문에 릿지나 라쏘와 같은 선형회귀를 통한 회귀는 효과적이지 않을 수 있다. 전처리가 부족했을 수 있다. 